In [11]:
from gatubelxs.page_views import generate_page_view
from gatubelxs.click_events import generate_click_event
from gatubelxs.user_interaction import generate_user_interaction
from kafka import KafkaProducer
from random import randint
import json
import time

In [12]:
KAFKA_SERVER = '5a7ab902bd93:9093'
TOPIC_PAGE_VIEWS = "page_views"
TOPIC_CLICK_EVENTS = "click_events"
TOPIC_USER_INTERACTIONS = "user_interactions"

NUMBER_SESSION = 250 #Number of web sessions to simulate
MAX_EVENTS_PER_SESSION = 10

In [13]:
#Producer
producer = KafkaProducer(
    bootstrap_servers=KAFKA_SERVER,
    value_serializer=lambda m: json.dumps(m).encode('utf-8')
)

In [14]:
import uuid
from random import randint

def simulate_session():
    user_id = str(uuid.uuid4())
    session_id = str(uuid.uuid4())

    n_page_views = randint(1, MAX_EVENTS_PER_SESSION)
    n_clicks = randint(0, MAX_EVENTS_PER_SESSION)
    n_interactions = randint(0, MAX_EVENTS_PER_SESSION)

    stages = ["browse", "cart", "checkout"]
    stage_treshholds = [0.6, 0.85]

    for _ in range(n_page_views):
        evt = generate_page_view(user_id, session_id)
        producer.send(TOPIC_PAGE_VIEWS, evt)
        print(f"[{TOPIC_PAGE_VIEWS}] ->", evt)
    
    for i in range(n_clicks):
        progress = i / max(n_clicks, 1)
        if progress < stage_treshholds[0]:
            stage = "browse"
        elif progress < stage_treshholds[1]:
            stage = "cart"
        else:
            stage = "checkout"
        evt = generate_click_event(user_id, session_id, user_stage=stage)
        producer.send(TOPIC_CLICK_EVENTS, evt)
        print(f"[{TOPIC_CLICK_EVENTS}] ->", evt)
    
    for i in range(n_interactions):
        evt = generate_user_interaction(user_id, session_id)
        producer.send(TOPIC_USER_INTERACTIONS, evt)
        print(f"[{TOPIC_USER_INTERACTIONS}] ->", evt)

In [15]:
try:
    print(f"Starting to produce sessions...")
    for i in range(NUMBER_SESSION):
        simulate_session()
        time.sleep(2)
except KeyboardInterrupt:
    print("Interrumpted by user; stopping.")
finally:
    producer.flush()
    producer.close()

Starting to produce sessions...
[page_views] -> {'user_id': 'e163e6c4-9dac-47be-b0ba-a8f8d9fd8c09', 'session_id': '352b9628-b3d0-48cb-9947-f8e3e2ae1c1e', 'page_url': '/product/1032', 'referrer_url': 'blog', 'category': 'toys', 'price': 393.23, 'timestamp': '2025-05-11T03:31:47.946595'}
[click_events] -> {'user_id': 'e163e6c4-9dac-47be-b0ba-a8f8d9fd8c09', 'session_id': '352b9628-b3d0-48cb-9947-f8e3e2ae1c1e', 'element_id': 'btn_add_to_cart', 'page_url': '/product/1173', 'category': 'home', 'price': 461.52, 'timestamp': '2025-05-11T03:31:47.950043', 'x_coord': 415.74, 'y_coord': 619.24}
[click_events] -> {'user_id': 'e163e6c4-9dac-47be-b0ba-a8f8d9fd8c09', 'session_id': '352b9628-b3d0-48cb-9947-f8e3e2ae1c1e', 'element_id': 'btn_add_to_cart', 'page_url': '/product/1179', 'category': 'electronics', 'price': 187.3, 'timestamp': '2025-05-11T03:31:48.052021', 'x_coord': 251.47, 'y_coord': 586.52}
[click_events] -> {'user_id': 'e163e6c4-9dac-47be-b0ba-a8f8d9fd8c09', 'session_id': '352b9628-b3d0-